In [29]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [31]:
file = pd.read_csv('diabetes_012_health_indicators_BRFSS2015.csv')

Y = file['Diabetes_012'].values
X = file.loc[:, file.columns!='Diabetes_012'].values
X_train, X_test, Y_train, Y_test = train_test_split( X, Y, test_size=0.2, random_state=20)

print(Y.shape)
print(X.shape)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(253680,)
(253680, 21)
(202944, 21)
(50736, 21)
(202944,)
(50736,)


In [8]:
file.head()

,Diabetes_012,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [ ]:
#determine which features are the most important with mutual information
#invent new features in several real-world problem domains
#encode high-cardinality categoricals with a target encoding
#create segmentation features with k-means clustering
#decompose a dataset's variation into features with principal component analysis